### SQL-запросы к базе данных EdTech-приложения

Образовательные курсы нашего приложения состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов (под усердным студентом мы понимаем студента, который хотя бы раз за текущий месяц правильно решил 20 задач).

В этом контексте нужно понять, имеется ли в виду решил ли человек вообще за весь месяц 20 и более горошин, либо в разбивке по предметам (и "хотя бы раз" в данном случае будет именно про то, что студент по одному из предметов решил 20 горошин. Вряд ли можно считать очень усердными тех студентов, которые по каждому предмету решили по, скажем, 3-5 горошин, набрав в сумме за месяц 20. Это если отталкиваться от логики. Если же отталкиваться лишь от поставленного ТЗ, то такой студент, конечно, тоже очень усердный. В любом случае, предоставляю решение в двух вариантах: с разбивкой по предметам и без нее.

Первое – без разбивки.

~~~~sql
SELECT 
    DATE_PART('year', pea_month)::INT AS year, 
    TO_CHAR(pea_month, 'Month') AS month,
    COUNT(DISTINCT st_id) AS hardworking_students
  FROM
    (SELECT
        DATE_TRUNC('month', timest)::DATE AS pea_month, st_id, COUNT(correct) AS correct_peas
       FROM peas
      WHERE correct = true
      GROUP BY pea_month, st_id) t1
 WHERE correct_peas >= 20
 GROUP BY pea_month
 ORDER BY 1, 2
~~~~

In [6]:
import pandas as pd
import requests

In [8]:
# importing query results from Redash with API

def redash_results(query_id):
    
    api_key = 'mzH6hFnPt8NoZVKREXaTok8VSE35rza4fQf1pqp4'
    
    redash_url = 'https://redash.lab.karpov.courses/api/queries/{}/results.json'.format(query_id)

    headers = {'Authorization': f'Key {api_key}'}

    response = requests.get(redash_url, headers = headers)

    if response.status_code == 200:
        data = response.json()
        results = data['query_result']['data']['rows']
        result = display(pd.DataFrame(results))
    else:
        result = print(f'Ошибка при получении данных: {response.status_code} - {response.text}')
    
    return result

redash_results(55879)

,year,month,hardworking_students
0,2021,October,136


Второе – с разбивкой по предметам (возьмем, к примеру, студента с айди 4557: он решил за месяц 19 горошин по статистике, 8 – по теории вероятностей и 17 по Python. В сумме у него, конечно, наберется 20 горошин, но по отдельности за каждый предмет он ни разу не решил правильно 20 задачек. В связи с этим из общего количества усердных студентов такой студент выпадет.)


~~~~sql
 SELECT 
    DATE_PART('year', pea_month)::INT AS year, 
    TO_CHAR(pea_month, 'Month') AS month, 
    COUNT(DISTINCT st_id) AS hardworking_students
 FROM
    (SELECT 
        DATE_TRUNC('month', timest)::DATE AS pea_month, 
        st_id, subject, 
        COUNT(correct) AS correct_peas
    FROM peas
    WHERE correct = true
    GROUP BY pea_month, st_id, subject) t2
 WHERE correct_peas >= 20
 GROUP BY pea_month
~~~~

In [12]:
# importing query results from Redash

redash_results(55907)

,year,month,hardworking_students
0,2021,October,78


Наша образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

- ARPU (считается относительно всех пользователей, попавших в группы)
- ARPAU 
- CR в покупку 
- СR активного пользователя в покупку 
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.
Активным по математике считается пользователь, за все время правильно решивший 2 или больше задач по математике.

~~~~sql

WITH active_students AS 
(
SELECT st_id
  FROM peas
 WHERE correct = true
 GROUP BY st_id
HAVING COUNT(correct) > 10
),
active_math_students AS 
(
SELECT st_id
  FROM peas
 WHERE correct = true AND subject = 'Math'
 GROUP BY st_id
HAVING COUNT(correct) >= 2
) 

SELECT
    test_grp,
    
    SUM(money) / COUNT(DISTINCT st_id) AS ARPPU,
    
    SUM(money) / COUNT(DISTINCT st_id) FILTER (WHERE st_id IN (SELECT * 
                                                                 FROM active_students)) AS ARPAU,
                                                                 
    COUNT(DISTINCT st_id) FILTER (WHERE sale_time IS NOT NULL) 
        / COUNT(DISTINCT st_id)::decimal AS CR,
        
    COUNT(DISTINCT st_id) FILTER (WHERE sale_time IS NOT NULL 
                                      AND st_id IN (SELECT *
                                                    FROM active_students)) 
        / COUNT(DISTINCT st_id) FILTER (WHERE st_id IN (SELECT *
                                                          FROM active_students))::decimal AS CR_active,
                                                          
    COUNT(DISTINCT st_id) FILTER (WHERE sale_time IS NOT NULL 
                                      AND st_id IN (SELECT *
                                                    FROM active_math_students) AND subject = 'Math') 
        / COUNT(DISTINCT st_id) FILTER (WHERE st_id IN (SELECT *
                                                          FROM active_math_students))::decimal AS CR_math_active
FROM studs 
LEFT JOIN final_project_check USING(st_id)
GROUP BY test_grp
~~~~

In [13]:
# importing query results from Redash

redash_results(55908)

,test_grp,arppu,arpau,cr,cr_active,cr_math_active
0,control,4540,10905,0.049180,0.110236,0.061224
1,pilot,11508,35364,0.108475,0.260417,0.095238


### Итого

- Количество очень усердных студентов составляет 136 (либо 78 - тут имеет смысл уточнить у заказчика ТЗ) человек.

- Видим, что конверсия в покупку в общем и в случае активных пользователей в тестовой группе в два раза выше, чем в контроле. Требуется дальнейший анализ для того, чтобы выяснить, значимы ли такие различия статистически.